In [1]:
import wandb

from utils.neural_network import NeuralNetwork
from utils.wandb_classes import WandbTrainer, WandbCallback
from utils.helper_functions import get_optimizer, load_data

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bullseye2608 (bullseye2608-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data('fashion_mnist')

### Trials

In [ ]:
nn = NeuralNetwork(layer_sizes=[784, 128, 128, 128, 10], 
                   activation_functions=['relu', 'relu', 'relu', 'softmax'], 
                   weight_init='xavier',
                   weight_decay=0.0, LOG_EACH=True)

H, A = nn.forward_propagation(X_train)
loss = nn.compute_loss(H[-1], y_train)
print(f'{nn.compute_accuracy(X_val, y_val) :>.6f}')

nn.set_optimizer({'name':'nadam', 'learning_rate':0.001, 'epsilon':1e-7, 'beta1':0.95})

0.125000


In [ ]:
num_trial_datapoints = 54000

LOG_EACH = True

nn.train(X_train[:num_trial_datapoints], 
         y_train[:num_trial_datapoints], 
         X_val, y_val, 
         batch_size=64, 
         num_epochs=5, 
         loss_type='cross_entropy', 
         log_every=5000)

LOG_EACH = False

print('--'*20,'DONE','--'*20)
print(nn.compute_accuracy(X_test, y_test))

Running NadamOptimizer self.learning_rate = 0.001 self.beta1 = 0.95 self.beta2 = 0.999 self.epsilon = 1e-07
Epoch 1/5, Iteration   0/844 --> Train Loss: 2.26874, Val Loss: 2.17015
---------------------------------------- DONE ----------------------------------------
0.8787


### WANDB SWEEP

In [ ]:
# # Create New sweep
# import yaml

# with open("sweep_config.yaml", "r") as file:
#         sweep_config = yaml.safe_load(file)

# sweep_id = wandb.sweep(sweep_config, 
#                        entity="bullseye2608-indian-institute-of-technology-madras",
#                        project="fashion_mnist_hp_search")

# # Run the sweep
# wandb.agent(sweep_id, wandb_sweep_helper_function, count=60)

In [ ]:
# # Continue the sweep

# sweep_id_cont = "bullseye2608-indian-institute-of-technology-madras/fashion_mnist_hp_search/vhbqpquu"
# trainer = WandbTrainer()

# wandb.agent(sweep_id_cont, trainer.train, count=10)

### WANDB RUNS

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime

from utils.helper_functions import OptimalConfig

In [ ]:


# # Set random seeds for reproducibility
# np.random.seed(42)

# # Fashion MNIST class names
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



# # Create and log a confusion matrix visualization
# def plot_confusion_matrix(y_true, y_pred, run_id):
#     cm = confusion_matrix(y_true, y_pred)
#     cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
#     # Create a DataFrame for better visualization
#     cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
#     cm_norm_df = pd.DataFrame(cm_normalized, index=class_names, columns=class_names)
    
#     # Create figure with normalized confusion matrix
#     plt.figure(figsize=(10, 8))
#     cmap = sns.diverging_palette(220, 20, as_cmap=True)
#     sns.heatmap(cm_norm_df, annot=True, fmt='.2f', cmap=cmap, 
#                 linewidths=.5, cbar_kws={"shrink": .8})
    
#     plt.title(f'Normalized Confusion Matrix - Run {run_id+1}', fontsize=16)
#     plt.ylabel('True Label', fontsize=12)
#     plt.xlabel('Predicted Label', fontsize=12)
    
#     # Calculate metrics
#     precision = np.diag(cm) / np.sum(cm, axis=0)
#     recall = np.diag(cm) / np.sum(cm, axis=1)
#     f1 = 2 * precision * recall / (precision + recall)
    
#     # Save and return the figure
#     cm_filename = f"confusion_matrix_run_{run_id+1}.png"
#     plt.savefig(cm_filename, dpi=300, bbox_inches='tight')
#     plt.close()
    
#     return cm_filename, cm, precision, recall, f1

# # Run the optimal model configuration multiple times
# def run_multiple_experiments(num_runs=17):
    
#     # Define optimal hyperparameter configuration
#     optimal_config = OptimalConfig()
    
#     # Lists to store results from all runs
#     all_metrics = []
#     all_cms = []
#     all_y_preds = []
#     run_ids = []
    
#     # Create a group ID for all runs
#     group_id = f"optimal-config-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
#     # Run the model multiple times
#     for run_id in range(num_runs):
#         # Set a different random seed for each run
#         np.random.seed(42 + run_id)
        
#         # Initialize a new wandb run
#         run = wandb.init(
#             project="fashion_mnist_hp_search",
#             name=f"optimal-run-{run_id+1}",
#             tags=["optimal-config"],
#             group=group_id,
#             config=optimal_config,
#         )
#         wandb.config.update(optimal_config)
        
#         config = optimal_config
        
#         layer_sizes = [784] + [config.hidden_size]*config.hidden_layers + [10]
#         activation_functions = [config.activation]*config.hidden_layers + ['softmax']
        
#         nn = NeuralNetwork(layer_sizes=layer_sizes, 
#                         activation_functions=activation_functions,
#                         weight_init=config.weight_init, 
#                         weight_decay=config.weight_decay)
        
#         wandb_callback = WandbCallback()
        
#         optimizer = get_optimizer(config.optimizer, config.learning_rate)
#         nn.set_optimizer(optimizer)
        
#         nn.train(
#             X_train,
#             y_train,
#             X_val,
#             y_val,
#             batch_size=config.batch_size,
#             num_epochs=config.epochs,
#             loss_type=config.loss,
#             log_every=1000,
#             callback=wandb_callback
#         )
        
#         test_accuracy = nn.compute_accuracy(X_test, y_test)
#         wandb.log({"test_accuracy": test_accuracy})
        
#         # Log the configuration
        
        
#         # Train the model
        
#         # Evaluate the model
#         test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=0)
        
#         # Get predictions
#         y_pred_probs = model.predict(x_test)
#         y_pred = np.argmax(y_pred_probs, axis=1)
        
#         # Store predictions
#         all_y_preds.append(y_pred)
        
#         # Create and log confusion matrix
#         cm_file, cm, precision, recall, f1 = plot_confusion_matrix(y_test_labels, y_pred, run_id)
#         all_cms.append(cm)
        
#         # Log confusion matrix image
#         wandb.log({"confusion_matrix": wandb.Image(cm_file)})
        
#         # Log interactive confusion matrix
#         wandb.log({"confusion_matrix_plot": wandb.plot.confusion_matrix(
#             probs=None,
#             y_true=y_test_labels,
#             preds=y_pred,
#             class_names=class_names
#         )})
        
#         # Log metrics
#         run_metrics = {
#             'test_loss': test_loss,
#             'test_accuracy': test_acc,
#             'run_id': run_id + 1
#         }
        
#         # Log per-class metrics
#         for i, class_name in enumerate(class_names):
#             run_metrics[f"precision_{class_name}"] = precision[i]
#             run_metrics[f"recall_{class_name}"] = recall[i]
#             run_metrics[f"f1_{class_name}"] = f1[i]
        
#         # Log all metrics
#         wandb.log(run_metrics)
        
#         # Store metrics for aggregate analysis
#         all_metrics.append(run_metrics)
#         run_ids.append(run.id)
        
#         # Finish the run
#         wandb.finish()
        
#         print(f"Completed run {run_id+1}/{num_runs} with accuracy: {test_acc:.4f}")
    
#     # Create an aggregate analysis run
#     run = wandb.init(
#         project="fashion-mnist-optimal",
#         name=f"aggregate-analysis-{num_runs}-runs",
#         tags=["aggregate", "analysis", group_id],
#         group=group_id
#     )
    
#     # Calculate average confusion matrix
#     avg_cm = np.mean(all_cms, axis=0)
#     std_cm = np.std(all_cms, axis=0)
    
#     # Create and log aggregate confusion matrix
#     plt.figure(figsize=(12, 10))
#     avg_cm_normalized = avg_cm.astype('float') / avg_cm.sum(axis=1)[:, np.newaxis]
#     avg_cm_df = pd.DataFrame(avg_cm_normalized, index=class_names, columns=class_names)
    
#     sns.heatmap(avg_cm_df, annot=True, fmt='.2f', cmap='viridis', 
#                 linewidths=.5, cbar_kws={"shrink": .8})
    
#     # Calculate aggregate metrics
#     accuracies = [m['test_accuracy'] for m in all_metrics]
#     mean_acc = np.mean(accuracies)
#     std_acc = np.std(accuracies)
    
#     plt.title(f'Aggregate Confusion Matrix (17 Runs)\nMean Accuracy: {mean_acc:.4f} ± {std_acc:.4f}', fontsize=16)
#     plt.ylabel('True Label', fontsize=12)
#     plt.xlabel('Predicted Label', fontsize=12)
    
#     # Save and log the aggregate confusion matrix
#     agg_cm_file = "aggregate_confusion_matrix.png"
#     plt.savefig(agg_cm_file, dpi=300, bbox_inches='tight')
#     plt.close()
    
#     wandb.log({"aggregate_confusion_matrix": wandb.Image(agg_cm_file)})
    
#     # Log aggregate metrics
#     agg_metrics = {
#         'mean_accuracy': mean_acc,
#         'std_accuracy': std_acc,
#         'min_accuracy': min(accuracies),
#         'max_accuracy': max(accuracies),
#         'num_runs': num_runs
#     }
    
#     # Create a summary table with links to all runs
#     run_table = wandb.Table(columns=["Run ID", "Accuracy", "Link"])
#     for i, (run_id, metrics) in enumerate(zip(run_ids, all_metrics)):
#         run_link = f"https://wandb.ai/[your-username]/fashion-mnist-optimal/runs/{run_id}"
#         run_table.add_data(i+1, metrics['test_accuracy'], run_link)
    
#     wandb.log({"runs_summary": run_table})
#     wandb.log(agg_metrics)
    
#     # Finish the aggregate run
#     wandb.finish()
    
#     return all_metrics, all_cms, all_y_preds

In [ ]:
# Fashion MNIST class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



# Create and log a confusion matrix visualization
def plot_confusion_matrix(y_true, y_pred, run_id):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create a DataFrame for better visualization
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    cm_norm_df = pd.DataFrame(cm_normalized, index=class_names, columns=class_names)
    
    # Create figure with normalized confusion matrix
    plt.figure(figsize=(10, 8))
    cmap = sns.diverging_palette(220, 20, as_cmap=True)
    sns.heatmap(cm_norm_df, annot=True, fmt='.2f', cmap=cmap, 
                linewidths=.5, cbar_kws={"shrink": .8})
    
    plt.title(f'Normalized Confusion Matrix - Run {run_id+1}', fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    
    # Calculate metrics
    precision = np.diag(cm) / np.sum(cm, axis=0)
    recall = np.diag(cm) / np.sum(cm, axis=1)
    f1 = 2 * precision * recall / (precision + recall)
    
    # Save and return the figure
    cm_filename = f"./confusion_matrices/confusion_matrix_run_{run_id+1}.png"
    plt.savefig(cm_filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    return cm_filename, cm, precision, recall, f1

# Run the optimal model configuration multiple times
def run_multiple_experiments():
    
    # Define optimal hyperparameter configuration
    
    # Lists to store results from all runs
    all_metrics = []
    all_cms = []
    all_y_preds = []
    run_ids = []
    
    # Create a group ID for all runs
    group_id = f"optimal-config-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # Set changes in runs
    variation = [
        {},
        {'num_layers': 4},
        {'batch_size': 32, 'num_layers': 4},
        {'epochs':15, 'batch_size': 128, 'num_layers': 5},
    ]
    num_runs = len(variation)
    
    # Run the model multiple times
    for run_id in range(num_runs):
        # Set a different random seed for each run
        np.random.seed(42 + run_id)
        config = OptimalConfig(**variation[run_id])
        
        # Initialize a new wandb run
        run = wandb.init(
            project="confusion_matrix_trials",
            name=f"optimal-run-{run_id+1}",
            tags=["optimal-config"],
            group=group_id,
            config=config,
        )
        wandb.config.update(config)
        config.print_config()
        
        
        layer_sizes = [784] + [config.hidden_size]*config.num_layers + [10]
        activation_functions = [config.activation]*config.num_layers + ['softmax']
        
        nn = NeuralNetwork(layer_sizes=layer_sizes, 
                        activation_functions=activation_functions,
                        weight_init=config.weight_init, 
                        weight_decay=config.weight_decay,
                        LOG_EACH=True)
        
        wandb_callback = WandbCallback()
        
        optimizer = get_optimizer(config.optimizer, config.learning_rate)
        nn.set_optimizer(optimizer)
        
        nn.train(
            X_train,
            y_train,
            X_val,
            y_val,
            batch_size=config.batch_size,
            num_epochs=config.epochs,
            loss_type=config.loss,
            log_every=1000,
            callback=wandb_callback
        )
        
        test_accuracy = nn.compute_accuracy(X_val, y_val)
        wandb.log({"test_accuracy": test_accuracy})
        
        # Evaluate the model
        test_loss = nn.compute_loss(nn.predict(X_val), y_val)
        test_acc = nn.compute_accuracy(X_val, y_val)
        
        # Get predictions
        y_pred_probs = nn.predict(X_val)
        y_pred = np.argmax(y_pred_probs, axis=1)
        
        # Store predictions
        all_y_preds.append(y_pred)
        
        # Create and log confusion matrix
        cm_file, cm, precision, recall, f1 = plot_confusion_matrix(y_val, y_pred, run_id)
        all_cms.append(cm)
        
        # Log confusion matrix image
        wandb.log({"confusion_matrix": wandb.Image(cm_file)})
        
        # Log interactive confusion matrix
        wandb.log({"confusion_matrix_plot": wandb.plot.confusion_matrix(
            probs=y_pred_probs,
            y_true=y_val,
            class_names=class_names
        )})
        
        # Log metrics
        run_metrics = {
            'test_loss': test_loss,
            'test_accuracy': test_acc,
            'run_id': run_id + 1
        }
    
        
        # Log all metrics
        wandb.log(run_metrics)
        
        # Store metrics for aggregate analysis
        all_metrics.append(run_metrics)
        run_ids.append(run.id)
        
        
        
        # TODO: Remove Boiler PLate code
        fig, cm, precision, recall, f1 = create_plotly_confusion_matrix(
            y_true=y_val,
            y_pred=y_pred,
            class_names=class_names,
            run_id=run_id
            )
        
        log_plotly_confusion_matrix_to_wandb(fig, run_id)
        
        
        
        # Finish the run
        wandb.finish()
        
        print(f"Completed run {run_id+1}/{num_runs} with accuracy: {test_acc:.4f}")
    
    # Create an aggregate analysis run
    run = wandb.init(
        project="confusion_matrix_trials",
        name=f"aggregate-analysis-{num_runs}-runs",
        tags=["aggregate", "analysis", group_id],
        group=group_id
    )
    
    # Calculate average confusion matrix
    avg_cm = np.mean(all_cms, axis=0)
    std_cm = np.std(all_cms, axis=0)
    
    # Create and log aggregate confusion matrix
    plt.figure(figsize=(12, 10))
    avg_cm_normalized = avg_cm.astype('float') / avg_cm.sum(axis=1)[:, np.newaxis]
    avg_cm_df = pd.DataFrame(avg_cm_normalized, index=class_names, columns=class_names)
    
    sns.heatmap(avg_cm_df, annot=True, fmt='.2f', cmap='viridis', 
                linewidths=.5, cbar_kws={"shrink": .8})
    
    # Calculate aggregate metrics
    accuracies = [m['test_accuracy'] for m in all_metrics]
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    plt.title(f'Aggregate Confusion Matrix (17 Runs)\nMean Accuracy: {mean_acc:.4f} ± {std_acc:.4f}', fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    
    # Save and log the aggregate confusion matrix
    agg_cm_file = "./confusion_matrices/"+"aggregate_confusion_matrix.png"
    plt.savefig(agg_cm_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    wandb.log({"aggregate_confusion_matrix": wandb.Image(agg_cm_file)})
    
    # Log aggregate metrics
    agg_metrics = {
        'mean_accuracy': mean_acc,
        'std_accuracy': std_acc,
        'min_accuracy': min(accuracies),
        'max_accuracy': max(accuracies),
        'num_runs': num_runs
    }
    
    # Create a summary table with links to all runs
    run_table = wandb.Table(columns=["Run ID", "Accuracy"])
    for i, (run_id, metrics) in enumerate(zip(run_ids, all_metrics)):
        run_table.add_data(i+1, metrics['test_accuracy'])
    
    wandb.log({"runs_summary": run_table})
    wandb.log(agg_metrics)
    
    # Finish the aggregate run
    wandb.finish()
    
    return all_metrics, all_cms, all_y_preds

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import wandb
import io

def create_plotly_confusion_matrix(y_true, y_pred, class_names, run_id=0):
    """
    Create an interactive confusion matrix using Plotly.js
    
    Parameters:
    -----------
    y_true : array-like
        Ground truth (correct) target values
    y_pred : array-like
        Estimated targets as returned by a classifier
    class_names : list
        List of class names (e.g., ['T-shirt', 'Trouser', ...])
    run_id : int
        Run identifier for multiple runs
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object of the confusion matrix
    cm : numpy.ndarray
        Raw confusion matrix
    """
    # Create confusion matrix with NumPy
    num_classes = len(class_names)
    cm = np.zeros((num_classes, num_classes), dtype=int)
    
    # Fill the confusion matrix
    for i in range(len(y_true)):
        true_class = y_true[i]
        pred_class = y_pred[i]
        cm[true_class, pred_class] += 1
    
    # Calculate normalized confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=cm_normalized,
        x=class_names,
        y=class_names,
        colorscale='Blues',
        text=cm,  # Show raw counts on hover
        texttemplate="%{text}",
        hovertemplate="True: %{y}<br>Predicted: %{x}<br>Count: %{text}<br>Rate: %{z:.2f}<extra></extra>",
    ))
    
    # Calculate accuracy
    accuracy = np.sum(np.diag(cm)) / np.sum(cm)
    
    # Add title and labels
    fig.update_layout(
        title={
            'text': f'Confusion Matrix - Run {run_id+1}<br><sup>Accuracy: {accuracy:.4f}</sup>',
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Predicted Label',
        yaxis_title='True Label',
        xaxis={'side': 'bottom'},
        width=800,
        height=800,
        font=dict(size=12)
    )
    
    # Add diagonal line effect
    diagonal_effect = []
    for i in range(len(class_names)):
        diagonal_effect.append(
            go.Scatter(
                x=[i-0.5, i+0.5],
                y=[i-0.5, i+0.5],
                mode='lines',
                line=dict(color='rgba(0,0,0,0.3)', width=1.5),
                showlegend=False,
                hoverinfo='none'
            )
        )
    
    # Add annotations for values
    annotations = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            # The text content
            text = f"{cm[i, j]}<br>({cm_normalized[i, j]:.2f})"
            
            # Text color should contrast with cell color
            text_color = 'white' if cm_normalized[i, j] > 0.5 else 'black'
            
            # Different styling for diagonal vs non-diagonal cells
            font_weight = 'bold' if i == j else 'normal'
            
            annotations.append(
                dict(
                    x=j,
                    y=i,
                    text=text,
                    font=dict(color=text_color, size=10, family='Arial', weight=font_weight),
                    showarrow=False,
                    align='center',
                )
            )
    
    for effect in diagonal_effect:
        fig.add_trace(effect)
    
    fig.update_layout(annotations=annotations)
    
    # Calculate per-class metrics
    precision = np.diag(cm) / np.sum(cm, axis=0)
    recall = np.diag(cm) / np.sum(cm, axis=1)
    f1 = 2 * precision * recall / (precision + recall)
    
    # Replace NaN values with 0
    precision = np.nan_to_num(precision)
    recall = np.nan_to_num(recall)
    f1 = np.nan_to_num(f1)
    
    return fig, cm, precision, recall, f1

def log_plotly_confusion_matrix_to_wandb(fig, run_id=0):
    """
    Log a Plotly confusion matrix to Weights & Biases
    
    Parameters:
    -----------
    fig : plotly.graph_objects.Figure
        Plotly figure to log
    run_id : int
        Run identifier
    
    Returns:
    --------
    html_path : str
        Path to the saved HTML file
    """
    
    print(f"Starting Logging confusion matrix for run {run_id+1}")
    # Save the figure as HTML and PNG
    html_path = f"confusion_matrix_run_{run_id+1}.html"
    png_path = f"confusion_matrix_run_{run_id+1}.png"
    
    # Save as interactive HTML
    pio.write_html(fig, file=html_path, auto_open=False)
    print('saved html file')
    
    # Save as static PNG for wandb image logging
    pio.write_image(fig, file=png_path)
    print('saved png file')
    
    # Log both versions to wandb
    wandb.log({
        "confusion_matrix_interactive": wandb.Html(html_path),
        "confusion_matrix_image": wandb.Image(png_path)
    })
    print('DONE')
    return html_path

# Example usage in your main script
def run_experiment_with_plotly_cm(run_id=0):
    # Your model training code here
    # ...
    
    # After getting predictions:
    fig, cm, precision, recall, f1 = create_plotly_confusion_matrix(
        y_true=y_test_labels,
        y_pred=y_pred,
        class_names=class_names,
        run_id=run_id
    )
    
    # Log to wandb
    html_path = log_plotly_confusion_matrix_to_wandb(fig, run_id)
    
    # Log additional metrics
    metrics = {
        'test_accuracy': accuracy,
        'run_id': run_id + 1
    }
    
    # Log per-class metrics
    # for i, class_name in enumerate(class_names):
    #     metrics[f"precision_{class_name}"] = precision[i]
    #     metrics[f"recall_{class_name}"] = recall[i]
    #     metrics[f"f1_{class_name}"] = f1[i]
    
    # wandb.log(metrics)
    
    return cm, precision, recall, f1

In [ ]:
metrics, cms, y_preds = run_multiple_experiments()
    
print("All runs completed!")
print(f"Average accuracy: {np.mean([m['test_accuracy'] for m in metrics]):.4f}")

epochs: 10
batch_size: 64
loss: cross_entropy
optimizer: adam
learning_rate: 0.001
momentum: 0.9
beta: 0.9
beta1: 0.9
beta2: 0.999
epsilon: 1e-08
weight_decay: 0
weight_init: xavier
num_layers: 3
hidden_size: 128
activation: relu
Running AdamOptimizer self.learning_rate = 0.001 self.beta1 = 0.9 self.beta2 = 0.999 self.epsilon = 1e-08
Epoch  1/10, Iteration   0/844 --> Train Loss: 2.32746, Val Loss: 2.20835
Epoch  2/10, Iteration 156/844 --> Train Loss: 0.32779, Val Loss: 0.45803
Epoch  3/10, Iteration 312/844 --> Train Loss: 0.37184, Val Loss: 0.36994
Epoch  4/10, Iteration 468/844 --> Train Loss: 0.33248, Val Loss: 0.33943
Epoch  5/10, Iteration 624/844 --> Train Loss: 0.30243, Val Loss: 0.32645
Epoch  6/10, Iteration 780/844 --> Train Loss: 0.36369, Val Loss: 0.30531
Epoch  8/10, Iteration  92/844 --> Train Loss: 0.18178, Val Loss: 0.30579
Epoch  9/10, Iteration 248/844 --> Train Loss: 0.27549, Val Loss: 0.30972
Epoch 10/10, Iteration 404/844 --> Train Loss: 0.21174, Val Loss: 0.3061